In [1]:
Output = c("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.2. Biomarker Distribution Analysis/Output")
cur_date = "100423"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(rlang)
library(PMCMRplus)

# reading in files
cytokine_df = data.frame(read_excel("Input/Processed_Cytokine_Data_081723.xlsx"))
mRNA_df = data.frame(read_excel("Input/Processed_mRNA_Data_081723.xlsx"))

# making some cols numeric
cytokine_df$Condensate_Conc = as.numeric(cytokine_df$Condensate_Conc)
mRNA_df$Concentration = as.numeric(mRNA_df$Concentration)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘rlang’


The following objects are masked from ‘package:purrr’:

    %@%, flatten, flatten_chr, flatten_dbl, flatten_int, flatten_lgl,
    flatten_raw, invoke, splice


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [2]:
head(cytokine_df)
head(mRNA_df)

,Subject_ID,Subject_No,Cytokine,Condensate,Burn_Condition,Condensate_Conc,Norm_Cytokine_Conc
,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,F_1,1,Eotaxin3,C,F,1,5.438924
2,F_1,1,Eotaxin3,C,F,25,3.630025
3,F_1,1,Eotaxin3,C,F,5,4.694272
4,F_1,1,Eotaxin3,C,S,1,6.165522
5,F_1,1,Eotaxin3,C,S,25,3.703871
6,F_1,1,Eotaxin3,C,S,5,4.562595


,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Norm_ddCT
,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,6,M_6,HMOX1,PBS,PBS,NA,24,3.912311
2,5,F_5,HMOX1,PBS,PBS,NA,4,3.067190
3,1,F_1,HMOX1,PBS,PBS,NA,4,3.843045
4,2,M_2,HMOX1,PBS,PBS,NA,4,3.415457
5,3,M_3,HMOX1,PBS,PBS,NA,4,3.234524
6,4,F_4,HMOX1,PBS,PBS,NA,4,3.294681


# Research Question: Are there statistically significant differences in each biomarker between burn conditions within the condensate samples?
Testing for statistical differences by comparing an individual biomarker's value (ie. concentration or ddCT) between burn conditions (ie. control, smoldering, and flaming) within each condensate (ie. plastic and cardboard) and time point (ie. 4, 24, 72).

A Friedman test is a non-paramteric alternative to an ANOVA. A Nemenyi post-hoc will be subsequently run. [ref](https://www.statology.org/friedman-test-r/)

# Friedman's Test

In [3]:
friedmans_burn_condition = function(df, concentration, value, biomarker, biomarker_name){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers, condensate, and
    # concentration. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: dataframe, concentration variable, value, biomarker name, biomarker variable name
    # :output: a dataframe containing the biomarker, variable name, condensate, concentration, time point, 
    # comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_biomarker = unique(df[[biomarker_name]])
    unique_condensate = c("C", "P")
    unique_conc = na.omit(unique(df[[concentration]]))

    # to prevent from having neg numbers in the denominator, the min value of the entire dataset will be added
    scale_num = abs(min(df[[value]]))
    
    friedmans_df = data.frame()
    nemenyis_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_biomarker)){
        for(j in 1:length(unique_condensate)){
            for(k in 1:length(unique_conc)){
                  
                # filtering for each biomarker, conc
                filtered_df = df %>%
                    filter(eval(rlang::parse_expr(biomarker_name)) == unique_biomarker[i], Condensate == "PBS" 
                           | Condensate == unique_condensate[j] &
                           eval(rlang::parse_expr(concentration)) == unique_conc[k])


                # friedman's
                friedmans_test = friedman.test(filtered_df[[value]], filtered_df$Burn_Condition, filtered_df$Subject_ID)
                friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)


                # Nemenyi's post hoc
                nemenyis_test = frdAllPairsNemenyiTest(filtered_df[[value]], filtered_df$Burn_Condition, filtered_df$Subject_ID)
                nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], 
                                   nemenyis_test$statistic[4])
                # calculating FC to get directionality
                control_df = filtered_df %>%
                    filter(Condensate == "PBS")
                smoldering_df = filtered_df %>%
                    filter(Burn_Condition == "S")
                flaming_df = filtered_df %>%
                    filter(Burn_Condition == "F")
                
                FC = c(log2((mean(flaming_df[[value]]) + scale_num)/(mean(control_df[[value]]) + scale_num)), # PBS vs. F
                       log2((mean(flaming_df[[value]]) + scale_num)/(mean(smoldering_df[[value]]) + scale_num)), # S vs. F
                       log2((mean(smoldering_df[[value]]) + scale_num)/(mean(control_df[[value]]) + scale_num))) # PBS vs. S
                
                nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], 
                                    nemenyis_test$p.value[4])
                nemenyis_comparison = c(paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                                rownames(nemenyis_test$statistic)[1]),
                                         paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                                rownames(nemenyis_test$statistic)[2]),
                                         paste0(colnames(nemenyis_test$statistic)[2], " vs. ", 
                                                rownames(nemenyis_test$statistic)[2]))


                # contains biomarker, biomarker name, condensate, conc, stat, and p value
                friedmans_vector = cbind(biomarker, unique_biomarker[i], unique_condensate[j], unique_conc[k], 
                                         24, friedmans_test$statistic, friedmans_test$p.value)
                nemenyis_matrix = cbind(biomarker, unique_biomarker[i], unique_condensate[j], unique_conc[k], 
                                        24, nemenyis_comparison, nemenyis_stats, FC, nemenyis_pvalues)
                friedmans_df = rbind(friedmans_df, friedmans_vector)
                nemenyis_df = rbind(nemenyis_df, nemenyis_matrix)
                
            }
        }
    }
    
    # adding col names
    colnames(friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Statistic", "P Value")
    colnames(nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Comparison", "Statistic", "FC", "P Value")
    
    # calculating padj values
    friedmans_df = friedmans_df %>%
        group_by(Condensate, Concentration) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))
    nemenyis_df = nemenyis_df %>%
        group_by(Condensate, Concentration) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

    
    return(list(friedmans_df, nemenyis_df))
}

# calling fn
cytokine_friedmans = friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", 
                                              "Cytokine", "Cytokine")

In [4]:
# the mRNA df doesn't consistently test all 3 concentrations (1,5, or 25 micrograms) or 
# time points(4,24,72) therefore we can't subset the df using a loop so we'll create separate dfs
# for each concentration and time point
split_mRNA_df = mRNA_df %>%
    group_by(Concentration, Time_Point) %>%
    group_split()

conc1_24_mRNA_df = split_mRNA_df[[1]]
conc1_72_mRNA_df = split_mRNA_df[[2]]
conc5_24_mRNA_df = split_mRNA_df[[3]]
conc5_72_mRNA_df = split_mRNA_df[[4]]
conc25_24_mRNA_df = split_mRNA_df[[5]]
conc25_4_mRNA_df = split_mRNA_df[[6]]
conc25_72_mRNA_df = split_mRNA_df[[7]]
concc_24_mRNA_df = split_mRNA_df[[8]]
concc_4_mRNA_df = split_mRNA_df[[9]]
concc_72_mRNA_df = split_mRNA_df[[10]]

# control samples have a concentration of NA, so they're all in a separate df
# adding them back into the other dataframes
conc1_24_mRNA_df = unique(rbind(conc1_24_mRNA_df, concc_24_mRNA_df))
conc1_72_mRNA_df = unique(rbind(conc1_72_mRNA_df, concc_72_mRNA_df))
conc25_4_mRNA_df = unique(rbind(conc25_4_mRNA_df, concc_4_mRNA_df))
conc25_24_mRNA_df = unique(rbind(conc25_24_mRNA_df, concc_24_mRNA_df))
conc25_72_mRNA_df = unique(rbind(conc25_72_mRNA_df, concc_72_mRNA_df))
conc5_24_mRNA_df = unique(rbind(conc5_24_mRNA_df, concc_24_mRNA_df))
conc5_72_mRNA_df = unique(rbind(conc5_72_mRNA_df, concc_72_mRNA_df))

# removing a sample I'll have to do separately: plastic, flaming, concentration 5, 24 hr exposure
five_subject_only_sample_df = conc5_24_mRNA_df %>%
    filter(Condensate == "P" & Burn_Condition == "F")
# adding back in controls
five_subject_only_sample_df = unique(rbind(five_subject_only_sample_df, concc_24_mRNA_df)) %>%
    # removing the extra subject that was added back in from the control df
    filter(Subject_ID != "M_6")
removed_conc5_24_mRNA_df = unique(rbind(anti_join(conc5_24_mRNA_df, five_subject_only_sample_df), concc_24_mRNA_df))

head(conc1_24_mRNA_df)

Joining with `by = join_by(Subject_No, Subject_ID, mRNA, Condensate,
Burn_Condition, Concentration, Time_Point, Norm_ddCT)`


Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Norm_ddCT
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
4,F_4,HMOX1,P,F,1,24,3.48011747
6,M_6,HMOX1,P,S,1,24,-0.06824032
1,F_1,HMOX1,C,F,1,24,4.63241245
1,F_1,HMOX1,P,F,1,24,1.49163928
1,F_1,HMOX1,P,S,1,24,2.47454559
2,M_2,HMOX1,C,F,1,24,2.43869489


In [5]:
removed_conc5_24_mRNA_df %>%
    filter(Burn_Condition == "PBS")

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Norm_ddCT
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
6,M_6,HMOX1,PBS,PBS,NA,24,3.912311
6,M_6,ALDH3A1,PBS,PBS,NA,24,3.715852
6,M_6,CXCL1,PBS,PBS,NA,24,3.626645
6,M_6,CXCR1,PBS,PBS,NA,24,-20.295235
6,M_6,GCLC,PBS,PBS,NA,24,3.864761
6,M_6,GCLM,PBS,PBS,NA,24,3.698636
6,M_6,GMCSF2,PBS,PBS,NA,24,-17.879558
6,M_6,IL1B,PBS,PBS,NA,24,3.770177
6,M_6,IL6,PBS,PBS,NA,24,-19.816319


In [6]:
friedmans_burn_condition_v2 = function(df, value, biomarker){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers and condensate. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: dataframe, value, biomarker variable name
    # :output: a dataframe containing the biomarker, variable name, condensate, concentration, time point, 
    # comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_biomarker = unique(df[[biomarker]])
    unique_condensate = c("C", "P")
    
    # to prevent from having neg numbers in the denominator, the min value of the entire dataset will be added
    scale_num = abs(min(df[[value]]))
    
    friedmans_df = data.frame()
    nemenyis_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_biomarker)){
        for(j in 1:length(unique_condensate)){
                  
            # filtering for each biomarker, conc
            filtered_df = df %>%
                filter(eval(rlang::parse_expr(biomarker)) == unique_biomarker[i], 
                       Condensate != unique_condensate[j])

            # friedman's
            friedmans_test = friedman.test(filtered_df[[value]], filtered_df$Burn_Condition, filtered_df$Subject_ID)
            friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)


            # Nemenyi's post hoc
            nemenyis_test = frdAllPairsNemenyiTest(filtered_df[[value]], filtered_df$Burn_Condition, filtered_df$Subject_ID)
            nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], 
                               nemenyis_test$statistic[4])
            
           # calculating FC to get directionality
            control_df = filtered_df %>%
                filter(Condensate == "PBS")
            smoldering_df = filtered_df %>%
                filter(Burn_Condition == "S")
            flaming_df = filtered_df %>%
                filter(Burn_Condition == "F")
                
            FC = c(log2((mean(flaming_df[[value]]) + scale_num)/(mean(control_df[[value]]) + scale_num)), # PBS vs. F
                   log2((mean(flaming_df[[value]]) + scale_num)/(mean(smoldering_df[[value]]) + scale_num)), # S vs. F
                   log2((mean(smoldering_df[[value]]) + scale_num)/(mean(control_df[[value]]) + scale_num))) # PBS vs. S
            
            nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], 
                                nemenyis_test$p.value[4])
            nemenyis_comparison = c(paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                            rownames(nemenyis_test$statistic)[1]),
                                     paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                            rownames(nemenyis_test$statistic)[2]),
                                     paste0(colnames(nemenyis_test$statistic)[2], " vs. ", 
                                            rownames(nemenyis_test$statistic)[2]))

            # contains biomarker, biomarker name, conc, time point, condensate, stat, and p value
             friedmans_vector = cbind(biomarker, unique_biomarker[i], unique_condensate[j], 
                                     unique(filtered_df$Concentration)[1], unique(filtered_df$Time_Point), friedmans_test$statistic,
                                     friedmans_test$p.value)
            nemenyis_matrix = cbind(biomarker, unique_biomarker[i], unique_condensate[j], 
                                     unique(filtered_df$Concentration)[1], unique(filtered_df$Time_Point),                               
                                     nemenyis_comparison, nemenyis_stats, FC, nemenyis_pvalues)
            friedmans_df = rbind(friedmans_df, friedmans_vector)
            nemenyis_df = rbind(nemenyis_df, nemenyis_matrix)

        }
    }
    
    # adding col names
    colnames(friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Statistic", "P Value")
    colnames(nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                               "Comparison", "Statistic", "FC", "P Value")
    
    
    return(list(friedmans_df, nemenyis_df))
}

# calling fn
conc1_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc1_24_mRNA_df, "Norm_ddCT", "mRNA")
conc1_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc1_72_mRNA_df, "Norm_ddCT", "mRNA")
conc25_4_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_4_mRNA_df, "Norm_ddCT", "mRNA")
conc25_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_24_mRNA_df, "Norm_ddCT", "mRNA")
conc25_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_72_mRNA_df, "Norm_ddCT", "mRNA")
removed_conc5_24_friedmans_mRNA_v2 = friedmans_burn_condition_v2(removed_conc5_24_mRNA_df, "Norm_ddCT", "mRNA")
conc5_72_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc5_72_mRNA_df, "Norm_ddCT", "mRNA")

In [7]:
# had to fix this df since these comparisons only contain cardboard control, smoldering
cardboard_df = removed_conc5_24_friedmans_mRNA_v2[[2]] %>%
    filter(Condensate == "C")

removed_FC = na.omit(as.numeric(cardboard_df$FC))
fixed_removed_cardboard_df = removed_conc5_24_friedmans_mRNA_v2[[2]] %>%
    select(-FC) %>%
    filter(Condensate == "C", `P Value` != "NA") %>%
    mutate(FC = removed_FC)

fixed_removed_df = rbind(fixed_removed_cardboard_df[,c(1:7,9,8)], removed_conc5_24_friedmans_mRNA_v2[[2]] %>%
    filter(Condensate == "P"))

head(fixed_removed_df)

,Biomarker,Variable Name,Condensate,Concentration,Time Point,Comparison,Statistic,FC,P Value
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,mRNA,HMOX1,C,5,24,PBS vs. S,1.15470053837925,0.920095754695201,0.414216178242521
2,mRNA,ALDH3A1,C,5,24,PBS vs. S,0,-0.0855225757846126,1
3,mRNA,CXCL1,C,5,24,PBS vs. S,0,-0.450818537020416,1
4,mRNA,CXCR1,C,5,24,PBS vs. S,0,-0.618897609599559,1
5,mRNA,GCLC,C,5,24,PBS vs. S,0,-0.287876814598008,1
6,mRNA,GCLM,C,5,24,PBS vs. S,0,-0.249946475374075,1


In [8]:
# having to go back to run friedmans on a sample that only has data for 5 subjects instead of 6
unique_biomarker = unique(five_subject_only_sample_df$mRNA)

# to prevent from having neg numbers in the denominator, the min value of the entire dataset will be added
scale_num = abs(min(five_subject_only_sample_df$Norm_ddCT))

five_subject_only_friedmans_df = data.frame()
five_subject_only_nemenyis_df = data.frame()
for (i in 1:length(unique_biomarker)){
    # filtering for each biomarker, conc
    filtered_df = five_subject_only_sample_df %>%
        filter(mRNA == unique_biomarker[i])

    # friedman's
    friedmans_test = friedman.test(filtered_df$Norm_ddCT, filtered_df$Burn_Condition, filtered_df$Subject_ID)
    friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)

    # Nemenyi's post hoc
    nemenyis_test = frdAllPairsNemenyiTest(filtered_df$Norm_ddCT, filtered_df$Burn_Condition, filtered_df$Subject_ID)
    nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], nemenyis_test$statistic[4])

   # calculating FC to get directionality
    control_df = filtered_df %>%
        filter(Condensate == "PBS")
    flaming_df = filtered_df %>%
        filter(Burn_Condition == "F")

    FC = c(log2((mean(flaming_df$Norm_ddCT) + scale_num)/(mean(control_df$Norm_ddCT) + scale_num))) # PBS vs. F

    nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], nemenyis_test$p.value[4])
    nemenyis_comparison = "PBS vs. F"

    # contains biomarker, biomarker name, conc, time point, condensate, stat, and p value
    friedmans_vector = cbind("mRNA", unique_biomarker[i], "P", 
                             unique(filtered_df$Concentration)[1], unique(filtered_df$Time_Point), friedmans_test$statistic,
                             friedmans_test$p.value)
    nemenyis_matrix = cbind("mRNA", unique_biomarker[i], "P", 
                             unique(filtered_df$Concentration)[1], unique(filtered_df$Time_Point),                               
                             nemenyis_comparison, nemenyis_stats, FC, nemenyis_pvalues)
    five_subject_only_friedmans_df = rbind(five_subject_only_friedmans_df, friedmans_vector)
    five_subject_only_nemenyis_df = rbind(five_subject_only_nemenyis_df, nemenyis_matrix)

}

# adding col names
colnames(five_subject_only_friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                           "Statistic", "P Value")
colnames(five_subject_only_nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Time Point", 
                           "Comparison", "Statistic", "FC", "P Value")

In [9]:
 # calculating padj values for mRNAs
mRNAs_friedmans_df = rbind(conc1_24_friedmans_mRNA_v2[[1]], conc1_72_friedmans_mRNA_v2[[1]], conc25_4_friedmans_mRNA_v2[[1]],
                conc25_24_friedmans_mRNA_v2[[1]], conc25_72_friedmans_mRNA_v2[[1]], removed_conc5_24_friedmans_mRNA_v2[[1]], 
                conc5_72_friedmans_mRNA_v2[[1]], five_subject_only_friedmans_df) %>%
    group_by(Condensate, Concentration) %>%
    mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))
mRNAs_nemenyis_df = rbind(conc1_24_friedmans_mRNA_v2[[2]], conc1_72_friedmans_mRNA_v2[[2]], conc25_4_friedmans_mRNA_v2[[2]],
                conc25_24_friedmans_mRNA_v2[[2]], conc25_72_friedmans_mRNA_v2[[2]], fixed_removed_df, 
                conc5_72_friedmans_mRNA_v2[[2]], five_subject_only_nemenyis_df) %>%
        # removing uneeded rows from five subject only complication
        filter(`P Value` != "NA") %>%
    group_by(Condensate, Concentration) %>%
    mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

In [10]:
# creating 1 df
friedmans_df = rbind(cytokine_friedmans[[1]], mRNAs_friedmans_df)
nemenyis_df = rbind(cytokine_friedmans[[2]], mRNAs_nemenyis_df)

head(friedmans_df)
head(nemenyis_df)

Biomarker,Variable Name,Condensate,Concentration,Time Point,Statistic,P Value,P Adj
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Cytokine,Eotaxin3,C,1,24,2.33333333333333,0.311403223914598,0.3781325
Cytokine,Eotaxin3,C,25,24,0.333333333333333,0.846481724890614,0.8993868
Cytokine,Eotaxin3,C,5,24,1.33333333333333,0.513417119032592,0.7931555
Cytokine,Eotaxin3,P,1,24,1.33333333333333,0.513417119032592,0.6234351
Cytokine,Eotaxin3,P,25,24,0.333333333333333,0.846481724890614,0.8464817
Cytokine,Eotaxin3,P,5,24,3,0.22313016014843,0.4741516


Biomarker,Variable Name,Condensate,Concentration,Time Point,Comparison,Statistic,FC,P Value,P Adj
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Cytokine,Eotaxin3,C,1,24,F vs. PBS,0.408248290463863,0.0709529313971492,0.955103094750503,1.0000000
Cytokine,Eotaxin3,C,1,24,F vs. S,1.63299316185545,-0.0289361721576894,0.480432575468159,0.9074838
Cytokine,Eotaxin3,C,1,24,PBS vs. S,2.04124145231932,0.0998891035548385,0.318645081847549,0.6771208
Cytokine,Eotaxin3,C,25,24,F vs. PBS,0.816496580927726,-0.0564554358159761,0.832229218116194,1.0000000
Cytokine,Eotaxin3,C,25,24,F vs. S,0.408248290463863,-0.0508804397160569,0.955103094750503,1.0000000
Cytokine,Eotaxin3,C,25,24,PBS vs. S,0.408248290463863,-0.00557499609991912,0.955103094750503,1.0000000


In [11]:
# exporting
write.xlsx(friedmans_df, paste0(Output,"/", "Friedmans_Results_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(nemenyis_df, paste0(Output,"/", "Nemenyis_Results_", cur_date, ".xlsx"), 
           rowNames = FALSE)